<a href="https://colab.research.google.com/github/baekjiyeon123/groupproject_b3/blob/main/foodtraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics
from IPython import display
import ultralytics

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
import torch

In [ ]:
%cd /content/drive/MyDrive/

!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd /content/drive/MyDrive/yolov5
%pip install -r requirements.txt
%pip install -q roboflow

%pip install --upgrade imageio
from IPython.display import Image, clear_output

In [ ]:
from tqdm import tqdm
import IProgress
from ipywidgets import IntProgress

In [ ]:
torch.cuda.get_arch_list()

In [ ]:
print('All set. Using PyTorch version %s with %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
%cd /content/drive/MyDrive
!mkdir datasets

In [ ]:
%cd /content/drive/MyDrive/datasets4

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QdXV6krknWcnORVwwGnr")
project = rf.workspace("yong331").project("sk-shielders-module-project-3_koreanfood_detecting")
version = project.version(3)
dataset = version.download("yolov5")

In [ ]:
%cat /content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/data.yaml

In [ ]:
from glob import glob

img_list = glob('/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train/images/*.jpg')

print(len(img_list))

In [ ]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

In [ ]:
with open('/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

with open('/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/data.yaml', 'r') as f:
  data = yaml.full_load(f)
  #data = yaml.load(f, Loader=yaml.FullLoader)

print(data)

data['train'] = '/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/train.txt'
data['val'] = '/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/val.txt'

with open('/content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

In [ ]:
!pip install wandb

In [ ]:
 def fitness(x):
     # Model fitness as a weighted combination of metrics
     w = [0.0, 0.0, 0.1, 0.9]  # weights for [P, R, mAP@0.5, mAP@0.5:0.95]
     return (x[:, :4] * w).sum(1)

# Training the yolov5 Model

In [ ]:
!python train.py --img 416 --batch 16 --epochs 200 --data /content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/data.yaml --weights yolov5s.pt --cache

In [ ]:
!python detect.py --weights ./content/drive/MyDrive/yolov5/runs/train/yolov5s_results4/weights/best.pt --conf 0.5 --source /content/drive/MyDrive/datasets/SK-Shielders-Module-Project-3_KoreanFOOD_Detecting-3/test/images

In [ ]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/yolov5/runs/train/yolov5s_results

# you have best.pt and last.pt inside runs\train\exp2\weights

In [ ]:
# Convert weights to fp16 TFLite model

!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --img 416

In [ ]:
#After running the above command, you will have best-fp16.tflite in runs\train\exp2\weights folder

# Lets run the created tflite model.

!python detect.py --weights runs\train\exp2\weights\best-fp16.tflite --img 416 --conf 0.25 --source runs/train/exp2/test_images

# Our model is working fine and results are stored in runs\detect\exp3

In [ ]:
#Convert weights to int8 TFLite model

#!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --int8 --img 416 --data data/coco128.yaml

# 3. Get the android folder from here
https://github.com/AarohiSingla/TFLite-Object-Detection-Android-App-Tutorial-Using-YOLOv5

Now Put TFLite models in assets folder of Android project, and change
And then open android/app/src/main/java/org/tensorflow/lite/examples/detection/tflite/DetectorFactory.java

Do the following changes:

Change the inputSize of image as per your dataset.

Change labelFilename according to the classes of the model

Then run the program in Android Studio.